In [30]:
import pandas as pd
from pathlib import Path
import csv
import plotly.graph_objects as go
import plotly.express as px
from yahoo_fin import stock_info as si

In [31]:
# Get ticker list
tickers_path = "Resources/tickers_list/tickers_list.csv"
with open(tickers_path) as csvfile:
    data = csv.reader(csvfile)
    next(data)
    tickers_list = [row[0] for row in data]

### Display Summary of filtered stocks (Maybe make a new df for easy scanning)

# Display tickers
print("List of the tickers")
print(tickers_list)

# Display summaries like cumulative return, closing price, volume, etc

# Display fundamentals like P/E ratio

# Combine all Dataframe

### We can delete some of the print statement from app.py after completing this part (Marked with #D)

List of the tickers
['ENPH', 'IT', 'CMG', 'RJF', 'IRM', 'WRB']


In [32]:
# If time allows
# Create function to display candlestick
for ticker in tickers_list:
    csvpath = f"Resources/{ticker}.csv"
    df = pd.read_csv(csvpath, index_col=0, infer_datetime_format=True, parse_dates=True).dropna()
    df = df.reset_index()

    fig = go.Figure(
        layout_title_text=ticker + ' (July 2020 - September 2022)'
    )
    
    fig.add_trace(
        go.Candlestick(
                x=df['index'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                name=ticker
        ))
    fig.add_trace(
        go.Scatter(
            x=df['index'],
            y=df['SMA 50'],
            mode='lines',
            name='SMA 50'
        )    
    )
    
    fig.add_trace(
        go.Scatter(
            x=df['index'],
            y=df['SMA 200'],
            mode='lines',
            name='SMA 200'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['index'],
            y=df['EMA 20'],
            mode='lines',
            name='EMA 20'
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df['index'],
            y=df['SMA 20'],
            mode='lines',
            name='SMA 20'
        )
    )
    
    fig.update_layout(xaxis_rangeslider_visible=False, xaxis_title ='Date', yaxis_title='Price', width=900, height=600)
    display(fig)
    quote = si.get_quote_table(ticker)
    display(quote)

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:295: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:302: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



{'1y Target Est': 278.99,
 '52 Week Range': '113.40 - 324.84',
 'Ask': '317.15 x 800',
 'Avg. Volume': 3480798.0,
 'Beta (5Y Monthly)': 1.49,
 'Bid': '316.10 x 1100',
 "Day's Range": '309.88 - 319.65',
 'EPS (TTM)': 1.43,
 'Earnings Date': 'Oct 24, 2022 - Oct 28, 2022',
 'Ex-Dividend Date': nan,
 'Forward Dividend & Yield': 'N/A (N/A)',
 'Market Cap': '42.836B',
 'Open': 313.0,
 'PE Ratio (TTM)': 221.14,
 'Previous Close': 318.01,
 'Quote Price': 316.2300109863281,
 'Volume': 3013952.0}

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:295: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:302: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



{'1y Target Est': 327.75,
 '52 Week Range': '221.39 - 369.00',
 'Ask': '316.32 x 1000',
 'Avg. Volume': 471995.0,
 'Beta (5Y Monthly)': 1.4,
 'Bid': '301.56 x 1000',
 "Day's Range": '296.28 - 303.12',
 'EPS (TTM)': 8.87,
 'Earnings Date': 'Oct 31, 2022 - Nov 04, 2022',
 'Ex-Dividend Date': 'Jul 19, 1999',
 'Forward Dividend & Yield': 'N/A (N/A)',
 'Market Cap': '23.734B',
 'Open': 296.28,
 'PE Ratio (TTM)': 34.15,
 'Previous Close': 300.08,
 'Quote Price': 302.8999938964844,
 'Volume': 456491.0}

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:295: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:302: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



...

{'1y Target Est': 118.71,
 '52 Week Range': '84.86 - 117.37',
 'Ask': '110.35 x 900',
 'Avg. Volume': 943206.0,
 'Beta (5Y Monthly)': 1.06,
 'Bid': '108.90 x 800',
 "Day's Range": '105.61 - 109.77',
 'EPS (TTM)': 7.01,
 'Earnings Date': 'Oct 26, 2022',
 'Ex-Dividend Date': 'Sep 30, 2022',
 'Forward Dividend & Yield': '1.36 (1.27%)',
 'Market Cap': '23.626B',
 'Open': 105.61,
 'PE Ratio (TTM)': 15.62,
 'Previous Close': 106.85,
 'Quote Price': 109.47000122070312,
 'Volume': 871380.0}

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:295: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:302: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



{'1y Target Est': 55.17,
 '52 Week Range': '41.67 - 58.61',
 'Ask': '56.57 x 1000',
 'Avg. Volume': 1725968.0,
 'Beta (5Y Monthly)': 0.78,
 'Bid': '55.99 x 800',
 "Day's Range": '54.14 - 56.19',
 'EPS (TTM)': 1.27,
 'Earnings Date': 'Nov 02, 2022 - Nov 07, 2022',
 'Ex-Dividend Date': 'Sep 14, 2022',
 'Forward Dividend & Yield': '2.47 (4.40%)',
 'Market Cap': '16.331B',
 'Open': 54.45,
 'PE Ratio (TTM)': 44.24,
 'Previous Close': 55.29,
 'Quote Price': 56.18000030517578,
 'Volume': 2707630.0}

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:295: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\yutak\Anaconda3\envs\dev\lib\site-packages\yahoo_fin\stock_info.py:302: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



{'1y Target Est': 77.55,
 '52 Week Range': '47.73 - 72.32',
 'Ask': '68.43 x 900',
 'Avg. Volume': 1229263.0,
 'Beta (5Y Monthly)': 0.62,
 'Bid': '67.62 x 900',
 "Day's Range": '66.31 - 67.65',
 'EPS (TTM)': 4.75,
 'Earnings Date': 'Oct 19, 2022 - Oct 24, 2022',
 'Ex-Dividend Date': 'Sep 23, 2022',
 'Forward Dividend & Yield': '0.40 (0.60%)',
 'Market Cap': '17.922B',
 'Open': 66.42,
 'PE Ratio (TTM)': 14.22,
 'Previous Close': 66.94,
 'Quote Price': 67.55999755859375,
 'Volume': 705631.0}